In [1]:
import pandas as pd

glucose = pd.read_csv("data/GLUCOSE_training_data_final.csv")

In [2]:
glucose.head()

,experiment_id,story_id,worker_id,submission_time_normalized,worker_quality_rating,selected_sentence_index,story,selected_sentence,1_specificNL,1_specificStructured,...,8_generalStructured,9_specificNL,9_specificStructured,9_generalNL,9_generalStructured,10_specificNL,10_specificStructured,10_generalNL,10_generalStructured,number_filled_in
0,e56c7c3e-4660-40fb-80d0-052d566d676a__4,e56c7c3e-4660-40fb-80d0-052d566d676a,19,20190930,3,4,It was bedtime at our house. Two of the three ...,Finally he becomes tired and falls asleep.,The third kid continues to get out of bed and...,{The third kid}_[subject] {continues}_[verb] {...,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,2
1,b03eba12-db74-4fce-a3cc-c743948ef8c2__4,b03eba12-db74-4fce-a3cc-c743948ef8c2,2,20200115,1,4,Sara was invited to the football game. She sho...,"At the end of the game, her team won.",Sara goes to a football game >Causes/Enabl...,{Sara }_[subject] {goes }_[verb] {to }_[prepos...,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,3
2,7e7aeb03-12d8-4f02-9097-c3273a692f20__3,7e7aeb03-12d8-4f02-9097-c3273a692f20,60,20191230,1,3,Kyle was leaving to pick up his sister from wo...,He decided to check his dirty clothes basket.,Kyle realized his keys were missing >Causes/En...,{Kyle}_[subject] {realized}_[verb] {his keys}_...,...,escaped,escaped,escaped,escaped,escaped,He decided to check his dirty clothes basket f...,{He}_[subject] {decided to check}_[verb] {his ...,Someone_A decided to check Something_A (that h...,{Someone_A}_[subject] {decided to check}_[verb...,5
3,a2bb14bd-9094-4dcc-93a5-bb93b1c308a5__4,a2bb14bd-9094-4dcc-93a5-bb93b1c308a5,142,20191203,2,4,MY DAUGHTER HAD TO DO A SCIENCE PROJECT. SO SH...,SO CHEWING GUM CAN HELP WITH YOUR CONCENTRATIO...,My daughter has a science project >Causes/En...,{My daughter}_[subject] {has }_[verb] {a }_[pr...,...,escaped,Chewing gum can help with concentration >Resul...,{Chewing gum}_[subject] {can help}_[verb] {wit...,Chewing gum can help with concentration >Resul...,{Chewing gum}_[subject] {can help}_[verb] {wit...,escaped,escaped,escaped,escaped,7
4,b8c68631-fa42-40a0-93c7-6739402e2038__4,b8c68631-fa42-40a0-93c7-6739402e2038,19,20191213,3,4,Vivian couldn't stop thinking about that man s...,Vivian spent the rest of her life chasing that...,escaped,escaped,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,2


In [3]:
glucose.columns

Index(['experiment_id', 'story_id', 'worker_id', 'submission_time_normalized',
       'worker_quality_rating', 'selected_sentence_index', 'story',
       'selected_sentence', '1_specificNL', '1_specificStructured',
       '1_generalNL', '1_generalStructured', '2_specificNL',
       '2_specificStructured', '2_generalNL', '2_generalStructured',
       '3_specificNL', '3_specificStructured', '3_generalNL',
       '3_generalStructured', '4_specificNL', '4_specificStructured',
       '4_generalNL', '4_generalStructured', '5_specificNL',
       '5_specificStructured', '5_generalNL', '5_generalStructured',
       '6_specificNL', '6_specificStructured', '6_generalNL',
       '6_generalStructured', '7_specificNL', '7_specificStructured',
       '7_generalNL', '7_generalStructured', '8_specificNL',
       '8_specificStructured', '8_generalNL', '8_generalStructured',
       '9_specificNL', '9_specificStructured', '9_generalNL',
       '9_generalStructured', '10_specificNL', '10_specificStructured

In [4]:
len(glucose.query("`1_specificNL` != 'escaped' and `2_generalNL` != 'escaped'"))

33073

In [5]:
len(glucose.query("`6_specificNL` != 'escaped' and `7_generalNL` != 'escaped'"))

30461

In [6]:
MASK_TOKEN = "<MASK>"

def mask_story(story_sentences, selected_sent_idx, dim=1):
    masked_story = []

    if dim <= 5:
        for i in range(selected_sent_idx-1):
            masked_story.append(story_sentences[i])
        
        masked_story.append(MASK_TOKEN)
        masked_story.append(f"*{story_sentences[selected_sent_idx]}*")

        for i in range(selected_sent_idx+1, len(story_sentences)):
            masked_story.append(story_sentences[i])
    else:
        for i in range(selected_sent_idx):
            masked_story.append(story_sentences[i])

        masked_story.append(f"*{story_sentences[selected_sent_idx]}*")
        masked_story.append(MASK_TOKEN)

        for i in range(selected_sent_idx+2, len(story_sentences)):
            masked_story.append(story_sentences[i])
    
    return masked_story


In [7]:
glucose.iloc[3]["story"]

'MY DAUGHTER HAD TO DO A SCIENCE PROJECT. SO SHE WANTED TO USE ME FOR A EXAMPLE. SO SHE GAVE ME A PIECE OF GUM AND 21 MATH PROBLEMS, I GOT 1 WRONG SHE TOLD ME TO SPIT OUT THE GUM, GAVE  21 MATH PROBLEMS, I GOT 4 WRONG SO CHEWING GUM CAN HELP WITH YOUR CONCENTRATION AFTER ALL.'

In [8]:
from utils import get_sents

mask_story(get_sents(glucose.iloc[0]["story"]), 0, dim=6)

/Users/mismayil/.pyenv/versions/3.9.7/envs/op/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['*It was bedtime at our house.*',
 '<MASK>',
 'The third is a trouble maker.',
 'For two hours he continues to get out of bed and want to play.',
 'Finally he becomes tired and falls asleep.']

In [27]:
import re
from tqdm import tqdm
from utils import get_sents

def build_cross_dim_data(data):
    cross_dim_data = []

    for i, row in tqdm(data.iterrows(), total=len(data)):
        story = row["story"]
        selected_sentence = row["selected_sentence"]
        if "****" in story:
            sentences = story.split("****")
        else:
            sentences = get_sents(story, by_sentence=selected_sentence, use_cache=False)
        specific_1 = row["1_specificNL"].strip()
        general_2 = row["2_generalNL"].strip()
        specific_6 = row["6_specificNL"].strip()
        general_7 = row["7_generalNL"].strip()

        selected_sent_idx = sentences.index(selected_sentence)

        if specific_1 != "escaped" and general_2 != "escaped":
            masked_story = mask_story(sentences, selected_sent_idx, dim=1)
            cross_dim_data.append({
                "model_input": " ".join(masked_story),
                "model_output": specific_1.split("****") if "****" in specific_1 else specific_1,
                "critique_output": general_2.split("****") if "****" in general_2 else general_2,
                "masked_sentence": sentences[selected_sent_idx-1] if selected_sent_idx > 0 else None,
                "model_dim": 1,
                "critique_dim": 2
            })
        
        if specific_6 != "escaped" and general_7 != "escaped":
            masked_story = mask_story(sentences, selected_sent_idx, dim=6)
            cross_dim_data.append({
                "model_input": " ".join(masked_story),
                "model_output": specific_6.split("****") if "****" in specific_6 else specific_6,
                "critique_output": general_7.split("****") if "****" in general_7 else general_7,
                "masked_sentence": sentences[selected_sent_idx+1] if selected_sent_idx < len(sentences)-1 else None,
                "model_dim": 6,
                "critique_dim": 7
            })
    return cross_dim_data

In [14]:
glucose_cross_dim_data = build_cross_dim_data(glucose)
len(glucose_cross_dim_data)

63534

In [15]:
import json

with open("large_outputs/glucose_cross_dim_data.json", "w") as f:
    json.dump(glucose_cross_dim_data, f, indent=2)

In [20]:
test_glucose = pd.read_csv("data/test_set_answer_key.csv")

In [26]:
test_glucose.iloc[0]["6_specificNL"]

'The sweater went on  sale >Causes/Enables> Meg bought the sweater****the sweater goes on sale >Causes/Enables> Meg can buy the sweater****The sweater  went  on  sale >Causes/Enables> Meg  bought  the sweater'

In [28]:
test_glucose = pd.read_csv("data/test_set_answer_key.csv")
test_glucose_cross_dim_data = build_cross_dim_data(test_glucose)

100%|██████████| 500/500 [00:00<00:00, 10072.05it/s]


In [29]:
len(test_glucose_cross_dim_data)

103

In [30]:
with open("large_outputs/test_glucose_cross_dim_data.json", "w") as f:
    json.dump(test_glucose_cross_dim_data, f, indent=2)

In [27]:
glucose.iloc[7]["story"]

"Alicia decided she needed to talk to Taylor. She was upset at her. When the girls talked they realized they shouldn't be friends. They gave one last hug and walked away. They both held back tears."

In [29]:
for col in ["story", "selected_sentence", "1_specificNL", "2_generalNL"]:
    print(glucose.iloc[7][col], end="\n\n")

Alicia decided she needed to talk to Taylor. She was upset at her. When the girls talked they realized they shouldn't be friends. They gave one last hug and walked away. They both held back tears.

They both held back tears.

She  doesn't want  to  cry  >Causes/Enables> She  holds back tears 

Someone_A feel(s) sad  >Motivates> Someone_A cries 



In [31]:
glucose_cross_dim_data

[{'model_input': "Lucy's grandpa had been napping. She was happy when he finally woke up. She asked for a ride to a friends house. <MASK> *She sat and watched as he ate his lunch.*",
  'model_output': 'Grandpa Is Hungry >Causes/Enables> Grandpa Ate Food',
  'critique_output': 'Someone_B feel(s) Hunger >Motivates> Someone_B Ate Something_A',
  'model_dim': 1,
  'critique_dim': 2},
 {'model_input': "Alicia decided she needed to talk to Taylor. She was upset at her. When the girls talked they realized they shouldn't be friends. <MASK> *They both held back tears.*",
  'model_output': "She  doesn't want  to  cry  >Causes/Enables> She  holds back tears",
  'critique_output': 'Someone_A feel(s) sad  >Motivates> Someone_A cries',
  'model_dim': 1,
  'critique_dim': 2},
 {'model_input': "Peter had a pet cat. It had a very long, fluffy tail. Peter's baby liked to grab it. *One day, he grabbed so hard, some fur game off.* <MASK>",
  'model_output': "The baby grabbed so hard some fur came off >Cau